
##Data preperation and formation of the teams,players contributions and needs data



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###  Combine Defender Datasets from Top 5 Leagues (2020–2024)

This script loads defender statistics from FBref for the seasons 2020–2024 across the top five European leagues (Premier League, La Liga, Serie A, Bundesliga, Ligue 1), filters relevant data, adds a season tag, and saves a single combined dataset for further analysis.


In [ ]:
import pandas as pd
import os

# === Setup paths ===
base_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/"
seasons = ["2020_2021", "2021_2022", "2022_2023", "2023_2024"]
top_5_leagues = ["PremierLeague", "LaLiga", "SerieA", "Bundesliga", "Ligue1"]
dfs = []

# === Loop and process each file ===
for season in seasons:
    file_path = f"{base_path}{season}/merged_fbref_dataset_{season}_def.csv"
    try:
        df = pd.read_csv(file_path)
        df["Season"] = season.replace("_", "-")
        df = df[df["Source_League"].isin(top_5_leagues)]  # Filter for top 5 leagues
        dfs.append(df)
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
    except Exception as e:
        print(f"⚠️ Error processing {file_path}: {e}")

# === Combine and save ===
if dfs:
    df_combined = pd.concat(dfs, ignore_index=True)
    output_path = os.path.join(base_path, "combined_defenders_2020_2024_top5.csv")
    df_combined.to_csv(output_path, index=False)
    print(f"✅ Saved combined file for top 5 leagues at:\n{output_path}")
else:
    print("❌ No data loaded.")


✅ Saved combined file for top 5 leagues at:
/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/combined_defenders_2020_2024_top5.csv


### Merge Defender Player Data with Team-Level Defensive Context (2020–2024)

This script enriches defender-level data from FBref by integrating team-level defensive statistics. It computes a normalized defensive strength score based on goals against (GA), merges it with player data using squad and season identifiers, and produces a context-aware dataset for defenders across the top five European leagues from 2020 to 2024.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# === Load player-level defender data ===
player_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/combined_defenders_2020_2024_top5.csv"
df_players = pd.read_csv(player_path)

# === Load team-level data (make sure you already combined them) ===
team_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_leagues_all_seasons_combined.csv"
df_teams = pd.read_csv(team_path)

# === Standardize Season format if needed ===
df_teams["Season"] = df_teams["Season"].astype(str).str.replace("_", "-")
df_players["Season"] = df_players["Season"].astype(str)

# === Select team-level features ===
team_features = [
    "Squad", "Season", "keeper_GA", "keeper_CS%", "league_Pts/MP","defense_90s",
    "defense_Tkl+Int", "defense_Blocks", "defense_Err","defense_TklW","defense_Def 3rd","defense_Mid 3rd","defense_Att 3rd","defense_Tkl%","defense_Clr","defense_Err","defense_TklW"
]
df_team_filtered = df_teams[team_features].copy()

# === Compute inverse GA and normalize ===
df_team_filtered = df_team_filtered[df_team_filtered["keeper_GA"] > 0]
df_team_filtered["Def_Strength_Score"] = 1 / df_team_filtered["keeper_GA"]

# Normalize this score for interpretability
scaler = MinMaxScaler()
df_team_filtered["Def_Strength_Score_Norm"] = scaler.fit_transform(
    df_team_filtered[["Def_Strength_Score"]]
)

# === Join with player data ===
df_merged = df_players.merge(df_team_filtered, on=["Squad", "Season"], how="left")

# === Save the final merged file ===
output_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/DF/defenders_with_teamcontext_2020_2024.csv"
df_merged.to_csv(output_path, index=False)
print(f"✅ Final file saved at:\n{output_path}")


✅ Final file saved at:
/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/defenders_with_teamcontext_2020_2024.csv


### Compute Defender Contribution Index with Team Context and Role Adjustment

This script calculates a normalized contribution index for each defender by integrating player-level statistics with team-level defensive context. Contribution is computed as a share of key defensive actions (e.g., tackles, interceptions) relative to team totals. The final index also adjusts for the player's role weight and team defensive strength (based on goals conceded), enabling fair comparison across squads and seasons (2020–2024). The result is exported for further analysis or optimization modeling.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# === Load datasets ===
player_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_defense_players.csv"
team_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_leagues_all_seasons_combined.csv"

df_players = pd.read_csv(player_path)
df_teams = pd.read_csv(team_path)

# === Standardize Season format ===
df_teams["Season"] = df_teams["Season"].astype(str).str.replace("_", "-")
df_players["Season"] = df_players["Season"].astype(str)


# === Select relevant team features ===
team_features = [
    "Squad", "Season", "keeper_GA", "defense_Tkl+Int", "defense_Blocks",
    "defense_TklW", "defense_Def 3rd", "defense_Mid 3rd", "defense_Att 3rd",
    "defense_Clr", "defense_Tkl%", "defense_Tkl", "defense_Int","defense_90s"
]
df_team_filtered = df_teams[team_features].copy()
df_team_filtered = df_team_filtered[df_team_filtered["keeper_GA"] > 0]

# === Compute normalized team defensive score ===
df_team_filtered["Team_Def_Score"] = 1 / df_team_filtered["keeper_GA"]
df_team_filtered["Team_Def_Score_Norm"] = MinMaxScaler().fit_transform(df_team_filtered[["Team_Def_Score"]])

# === Merge team context into player data ===
df_merged = df_players.merge(df_team_filtered, on=["Squad", "Season"], how="left")

# === Map player feature -> corresponding team feature ===
feature_mapping = {
    "Tkl": "defense_Tkl",
    "TklW": "defense_TklW",
    "Int": "defense_Int",
    "Blocks": "defense_Blocks",
    "Clr": "defense_Clr",
    "Def 3rd": "defense_Def 3rd",
    "Mid 3rd": "defense_Mid 3rd",
     "Att 3rd": "defense_Att 3rd",
}

# === Compute raw contribution ratios ===
feature_contribs = []
for player_feat, team_feat in feature_mapping.items():
    contrib_col = f"{player_feat}_Contribution"
    df_merged[contrib_col] = df_merged[player_feat] / df_merged[team_feat]
    df_merged[contrib_col] = df_merged[contrib_col].replace([float('inf'), -float('inf')], 0).fillna(0)
    feature_contribs.append(contrib_col)

# === Sum raw contributions ===
df_merged["Total_Player_Contribution_def"] = df_merged[feature_contribs].sum(axis=1)

# === Normalize total contribution (once, after sum) ===
df_merged["Total_Player_Contribution_def_Norm"] = MinMaxScaler().fit_transform(
    df_merged[["Total_Player_Contribution_def"]]
)

# === Compute Role Adjustment for position-aware weighting ===
role_weights = {"DF": 1.0, "MF": 0.2, "FW": 0.2}

def compute_role_weight(pos_str):
    if pd.isna(pos_str):
        return 0.5
    roles = set(pos_str.replace("/", " ").replace(",", " ").split())
    weights = [role_weights.get(role.strip(), 0.5) for role in roles if role.strip() in role_weights]
    return sum(weights) / len(weights) if weights else 0.5

df_merged["Role_Adjustment"] = df_merged["Pos"].apply(compute_role_weight)

# === Final contribution index with normalized total ===
df_merged["Final_Contribution_Index"] = (
    df_merged["Total_Player_Contribution_def_Norm"] *
    df_merged["Team_Def_Score_Norm"] *
    df_merged["Role_Adjustment"]
)

# === Save to new CSV ===
output_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/DF/defenders_contribution_2020_2024.csv"
df_merged.to_csv(output_path, index=False)
print(f"\n✅ Saved final contribution data (scaled correctly) to:\n{output_path}")



✅ Saved final contribution data (scaled correctly) to:
/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/DF/defenders_contribution_2020_2024.csv


### Display Top Defenders (2023–2024) by Final Contribution Index

This snippet filters the previously computed contribution dataset to isolate the 2023–2024 season. It ranks players by their `Final_Contribution_Index`, which combines individual performance, team defensive context, and positional role relevance. The top 20 defenders are printed with key indicators, allowing for identification of high-impact performers in the most recent season.


In [ ]:
# === Filter for 2023–2024 season only ===
season_2324 = df_merged[df_merged["Season"] == "2023-2024"]

# === Sort by Final Contribution Index descending ===
top_players_2324 = season_2324.sort_values(by="Final_Contribution_Index", ascending=False)

# === Select columns to show ===
columns_to_show = ["Player", "Squad", "Pos", "Total_Player_Contribution_def", "Team_Def_Score_Norm", "Role_Adjustment", "Final_Contribution_Index"]

# === Display top N players (e.g. top 20) ===
top_n = 20
print(f"\n🏆 Top {top_n} Players for Season 2023–2024 by Final Contribution Index:\n")
print(top_players_2324[columns_to_show].head(top_n).to_string(index=False))



🏆 Top 20 Players for Season 2023–2024 by Final Contribution Index:

                Player      Squad Pos  Total_Player_Contribution_def  Team_Def_Score_Norm  Role_Adjustment  Final_Contribution_Index
           Melvin Bard       Nice  DF                       0.998916             0.615764              1.0                  0.405298
         Álex Grimaldo Leverkusen  DF                       0.767035             0.793651              1.0                  0.401122
        Matteo Darmian      Inter  DF                       0.678003             0.887446              1.0                  0.396465
    Alessandro Bastoni      Inter  DF                       0.643247             0.887446              1.0                  0.376141
     Jean-Clair Todibo       Nice  DF                       0.906174             0.615764              1.0                  0.367669
        Gleison Bremer   Juventus  DF                       0.971417             0.560676              1.0                  0.358880


### Computing Midfielder Contribution Index (2020–2024)

This script computes a custom `Final_Contribution_Index` for midfield players using passing and creative performance metrics. It combines player-level statistics with team-level passing data from top leagues across four seasons (2020–2024).

Key steps include:
- Mapping individual stats (e.g., xAG, Assists, Key Passes) to their team equivalents.
- Computing each player’s relative contribution as a proportion of their team’s totals.
- Creating a team-level score (`Team_Mid_Score`) from normalized metrics like Progressive Passes and Expected Assists.
- Weighting the player’s normalized contribution by their team context and positional relevance.

The result is a contribution index that reflects both individual creativity and tactical importance within the team context.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# === Load datasets ===
player_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_passing_players.csv"
team_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_leagues_all_seasons_combined.csv"

df_players = pd.read_csv(player_path)
df_teams = pd.read_csv(team_path)

# === Standardize Season format ===
df_teams["Season"] = df_teams["Season"].astype(str).str.replace("_", "-")
df_players["Season"] = df_players["Season"].astype(str)

# === Select relevant player and team features ===
player_features = [
    "xAG",  "Ast", "KP",
     "PrgP","PPA","1/3"
]

team_feature_mapping = {
    "xAG": "passing_xAG",
    "Ast": "passing_Ast",
    "KP": "passing_KP",
    "PPA": "passing_PPA",
    "PrgP": "passing_PrgP",

}

# === Prepare team dataframe with needed features ===
team_features = ["Squad", "Season"] + list(team_feature_mapping.values())
df_team_filtered = df_teams[team_features].copy()

# === Normalize individual team features for team scoring ===
team_score_components = ["passing_PrgP", "passing_KP","passing_PPA" ,"passing_xAG"]
for col in team_score_components:
    norm_col = f"Norm_{col}"
    df_team_filtered[norm_col] = MinMaxScaler().fit_transform(df_team_filtered[[col]])

# === Sum normalized components for final team score ===
df_team_filtered["Team_Mid_Score"] = (
    df_team_filtered["Norm_passing_PrgP"] +
    df_team_filtered["Norm_passing_KP"] +
    df_team_filtered["Norm_passing_xAG"]+
    df_team_filtered["Norm_passing_PPA"]
    )
df_team_filtered["Team_Mid_Score_Norm"] = MinMaxScaler().fit_transform(
    df_team_filtered[["Team_Mid_Score"]]
)

# === Merge with player data ===
df_merged = df_players.merge(df_team_filtered, on=["Squad", "Season"], how="left")

# === Compute raw contribution ratios ===
feature_contribs = []
for player_feat, team_feat in team_feature_mapping.items():
    contrib_col = f"{player_feat}_Contribution"
    df_merged[contrib_col] = df_merged[player_feat] / df_merged[team_feat]
    df_merged[contrib_col] = df_merged[contrib_col].replace([float('inf'), -float('inf')], 0).fillna(0)
    feature_contribs.append(contrib_col)

# === Sum raw contributions ===
df_merged["Total_Player_Contribution_mid"] = df_merged[feature_contribs].sum(axis=1)

# === Normalize total contribution ===
df_merged["Total_Player_Contribution_mid_Norm"] = MinMaxScaler().fit_transform(
    df_merged[["Total_Player_Contribution_mid"]]
)

# === Compute Role Adjustment ===
role_weights = {"DF": 0.6, "MF": 1.0, "FW": 0.7}

def compute_role_weight(pos_str):
    if pd.isna(pos_str):
        return 0.5
    roles = set(pos_str.replace("/", " ").replace(",", " ").split())
    weights = [role_weights.get(role.strip(), 0.5) for role in roles if role.strip() in role_weights]
    return sum(weights) / len(weights) if weights else 0.5

df_merged["Role_Adjustment"] = df_merged["Pos"].apply(compute_role_weight)

# === Final contribution index ===
df_merged["Final_Contribution_Index"] = (
    df_merged["Total_Player_Contribution_mid_Norm"] *
    df_merged["Team_Mid_Score_Norm"] *
    df_merged["Role_Adjustment"]
)

# === Save output ===
output_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/MF/midfielders_contribution_2020_2024.csv"
df_merged.to_csv(output_path, index=False)
print(f"\n✅ Saved midfielder contribution data to:\n{output_path}")



✅ Saved midfielder contribution data to:
/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/MF/midfielders_contribution_2020_2024.csv


### Extracting Top 20 Midfielders – 2023/2024 Season

This script filters the previously computed midfielder dataset to include only players from the 2023–2024 season. It then sorts them based on the `Final_Contribution_Index` and displays the top 20 performers.

The output includes:
- Player name
- Squad affiliation
- Season
- Playing position
- Final normalized contribution index

This summary helps highlight the most impactful midfielders of the season based on role-adjusted, context-aware contribution metrics.


In [ ]:
# === Filter for 2023–2024 season only ===
df_filtered = df_merged[df_merged["Season"] == "2023-2024"]

# === Show top 20 players by Final Contribution Index ===
top_20 = df_filtered.sort_values(by="Final_Contribution_Index", ascending=False)[
    ["Player", "Squad", "Season", "Pos", "Final_Contribution_Index"]
].head(20)

print("\n🏆 Top 20 Midfielders by Final Contribution Index (2023–2024):\n")
print(top_20.to_string(index=False))



🏆 Top 20 Midfielders by Final Contribution Index (2023–2024):

         Player           Squad    Season   Pos  Final_Contribution_Index
Martin Ødegaard         Arsenal 2023-2024    MF                  0.536970
 İlkay Gündoğan       Barcelona 2023-2024    MF                  0.376676
  Florian Wirtz      Leverkusen 2023-2024    MF                  0.375479
Kevin De Bruyne Manchester City 2023-2024    MF                  0.352394
Bruno Fernandes  Manchester Utd 2023-2024 MF,FW                  0.343585
          Rodri Manchester City 2023-2024    MF                  0.340845
 James Maddison       Tottenham 2023-2024    MF                  0.332358
    Pascal Groß        Brighton 2023-2024 MF,DF                  0.327715
    Xavi Simons      RB Leipzig 2023-2024    MF                  0.319769
Bruno Guimarães   Newcastle Utd 2023-2024    MF                  0.303435
    Declan Rice         Arsenal 2023-2024    MF                  0.298670
    Cole Palmer         Chelsea 2023-2024 FW,MF 

### Computing Forward Contribution Index (2020–2024)

This script evaluates attacking player performance by combining individual shooting metrics with team-level offensive context. The pipeline includes:

- Loading player and team datasets.
- Mapping player-level features (e.g., `Gls`, `xG`, `SoT`) to corresponding team-level stats.
- Computing per-player contribution ratios by dividing player stats by team totals.
- Normalizing total contributions across all players.
- Adjusting for player roles (e.g., FW = 1.0, MF = 0.7, DF = 0.5) to reflect positional impact.
- Combining individual contribution, team attacking score, and role weight into a final `Final_Contribution_Index`.

This index enables fair cross-team comparisons of forward effectiveness over the 2020–2024 seasons. The output CSV can be used to rank players and identify high-impact attackers within their tactical and statistical context.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# === Load datasets ===
player_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_shooting_players.csv"
team_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/all_leagues_all_seasons_combined.csv"

df_players = pd.read_csv(player_path)
df_teams = pd.read_csv(team_path)

# === Standardize Season format ===
df_teams["Season"] = df_teams["Season"].astype(str).str.replace("_", "-")
df_players["Season"] = df_players["Season"].astype(str)

# === Select relevant player and team features ===
player_features = [
    "Gls", "Sh", "SoT", "xG", "npxG", "G-xG", "np:G-xG"
]

team_feature_mapping = {
    "Gls": "standard_Gls",
    "Sh": "shooting_Sh",
    "SoT": "shooting_SoT",
    "xG": "shooting_xG",
    "npxG": "shooting_npxG",
    "G-xG": "shooting_G-xG",

}

# === Prepare team dataframe with needed features ===
team_features = ["Squad", "Season"] + list(team_feature_mapping.values())
df_team_filtered = df_teams[team_features].copy()

# === Normalize individual team features for team scoring ===
team_score_components = ["standard_Gls", "shooting_xG"]
for col in team_score_components:
    norm_col = f"Norm_{col}"
    df_team_filtered[norm_col] = MinMaxScaler().fit_transform(df_team_filtered[[col]])

# === Sum normalized components for final team score ===
df_team_filtered["Team_Att_Score"] = (
    df_team_filtered["Norm_standard_Gls"] +
    df_team_filtered["Norm_shooting_xG"]
)
df_team_filtered["Team_Att_Score_Norm"] = MinMaxScaler().fit_transform(
    df_team_filtered[["Team_Att_Score"]]
)

# === Merge with player data ===
df_merged = df_players.merge(df_team_filtered, on=["Squad", "Season"], how="left")

# === Compute raw contribution ratios ===
feature_contribs = []
for player_feat, team_feat in team_feature_mapping.items():
    contrib_col = f"{player_feat}_Contribution"
    df_merged[contrib_col] = df_merged[player_feat] / df_merged[team_feat]
    df_merged[contrib_col] = df_merged[contrib_col].replace([float('inf'), -float('inf')], 0).fillna(0)
    feature_contribs.append(contrib_col)

# === Sum raw contributions ===
df_merged["Total_Player_Contribution_att"] = df_merged[feature_contribs].sum(axis=1)

# === Normalize total contribution ===
df_merged["Total_Player_Contribution_att_Norm"] = MinMaxScaler().fit_transform(
    df_merged[["Total_Player_Contribution_att"]]
)

# === Compute Role Adjustment ===
role_weights = {"DF": 0.5, "MF": 0.7, "FW": 1}

def compute_role_weight(pos_str):
    if pd.isna(pos_str):
        return 0.5
    roles = set(pos_str.replace("/", " ").replace(",", " ").split())
    weights = [role_weights.get(role.strip(), 0.5) for role in roles if role.strip() in role_weights]
    return sum(weights) / len(weights) if weights else 0.5

df_merged["Role_Adjustment"] = df_merged["Pos"].apply(compute_role_weight)

# === Final contribution index ===
df_merged["Final_Contribution_Index"] = (
    df_merged["Total_Player_Contribution_att_Norm"] *
    df_merged["Team_Att_Score_Norm"] *
    df_merged["Role_Adjustment"]
)

# === Save output ===
output_path = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/FW/forwards_contribution_2020_2024.csv"
df_merged.to_csv(output_path, index=False)
print(f"\n✅ Saved forward contribution data to:\n{output_path}")


✅ Saved forward contribution data to:
/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/FW/forwards_contribution_2020_2024.csv


# === Filter for 2023–2024 season only ===
df_filtered = df_merged[df_merged["Season"] == "2023-2024"]

# === Show top 20 players by Final Contribution Index ===
top_20 = df_filtered.sort_values(by="Final_Contribution_Index", ascending=False)[
    ["Player", "Squad", "Season", "Pos", "Final_Contribution_Index", "Team_Att_Score"]
].head(20)

print("\n🏆 Top 20 Forwards by Final Contribution Index (2023–2024):\n")
print(top_20.to_string(index=False))


In [ ]:
# === Filter for 2023–2024 season only ===
df_filtered = df_merged[df_merged["Season"] == "2023-2024"]

# === Show top 20 players by Final Contribution Index ===
top_20 = df_filtered.sort_values(by="Final_Contribution_Index", ascending=False)[
    ["Player", "Squad", "Season", "Pos", "Final_Contribution_Index","Team_Att_Score"]
].head(20)

print("\n🏆 Top 20 Midfielders by Final Contribution Index (2023–2024):\n")
print(top_20.to_string(index=False))



🏆 Top 20 Midfielders by Final Contribution Index (2023–2024):

             Player           Squad    Season Pos  Final_Contribution_Index  Team_Att_Score
         Harry Kane   Bayern Munich 2023-2024  FW                  0.562269        1.876103
     Kingsley Coman   Bayern Munich 2023-2024  FW                  0.539075        1.876103
         Leroy Sané   Bayern Munich 2023-2024  FW                  0.539074        1.876103
Matteo Perez Vinlöf   Bayern Munich 2023-2024  FW                  0.536646        1.876103
Jonathan Asp Jensen   Bayern Munich 2023-2024  FW                  0.536646        1.876103
   Ryan Gravenberch   Bayern Munich 2023-2024  FW                  0.536646        1.876103
     Erling Haaland Manchester City 2023-2024  FW                  0.530613        1.813801
       Darwin Núñez       Liverpool 2023-2024  FW                  0.515632        1.753013
      Mohamed Salah       Liverpool 2023-2024  FW                  0.514341        1.753013
          Luis D

##  Merging and Scaling Player and Team-Level Datasets (2020–2024)

This script combines position-specific player contribution files — Forwards (FW), Midfielders (MF), and Defenders (DF) — with corresponding team-level contextual data across the top five European leagues from seasons 2020–2024.

###  Key Steps

- **Merge Player Contributions**  
  Combines FW, MF, and DF datasets using `Player` and `Season` as merge keys.
  
- **Aggregate Team Context**  
  Averages team-level statistics by `Squad` and `Season` across all three datasets.

- **Feature Scaling**  
  - Player features scaled to a range of **[20, 100]** using `MinMaxScaler` for uniformity.
  - Team features also scaled using the same range to ensure compatibility in optimization and similarity modeling.

- **Data Cleaning**  
  Removes duplicate columns after merge, fills missing data, and filters numeric features for scaling.

###  Output Files

- `merged_players.csv`  
  Contains all player-level features and contribution indexes across positions, merged and scaled.

- `merged_teams.csv`  
  Includes team-level statistics (e.g., defensive and passing metrics), normalized for use in similarity and optimization models.

###  Usage

These files serve as a clean and scalable base for:
- Player-team similarity analysis  
- Tactical role identification  
- Transfer optimization modeling



In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# === FILE PATHS ===
fw_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/FW/forwards_contribution_2020_2024.csv"
mf_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/MF/midfielders_contribution_2020_2024.csv"
df_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/fbref/DF/defenders_contribution_2020_2024.csv"

# === MERGE KEYS ===
player_merge_keys = ["Player", "Season"]
team_merge_keys = ["Squad", "Season"]

# === FEATURES ===
player_features = [
    "Age", "Tkl_Contribution", "TklW_Contribution", "Int_Contribution", "Squad",
    "Blocks_Contribution", "Clr_Contribution", "Def 3rd_Contribution",
    "Mid 3rd_Contribution", "Att 3rd_Contribution", "Total_Player_Contribution_def",
    "xAG_Contribution", "Ast_Contribution", "KP_Contribution", "PPA_Contribution", "PrgP_Contribution",
    "Total_Player_Contribution_mid", "Gls_Contribution", "Sh_Contribution", "SoT_Contribution",
    "xG_Contribution", "G-xG_Contribution", "Total_Player_Contribution_att", "Pos",
    "Team_Def_Score", "Team_Mid_Score", "Team_Att_Score"
]

team_features = [
    "defense_Tkl+Int", "defense_Blocks", "defense_TklW", "defense_Def 3rd", "defense_Mid 3rd",
    "defense_Att 3rd", "defense_Clr", "defense_Tkl", "defense_Int", "Team_Def_Score",
    "passing_xAG", "passing_Ast", "passing_KP", "passing_PPA", "passing_PrgP", "Team_Mid_Score",
    "standard_Gls", "shooting_Sh", "shooting_SoT", "shooting_xG", "shooting_G-xG", "Team_Att_Score"
]

# === LOAD FUNCTION ===
def load_filtered(file, keys, features):
    df = pd.read_csv(file)
    return df[[col for col in keys + features if col in df.columns]].copy()

# === LOAD AND MERGE PLAYERS ===
df_fw_p = load_filtered(fw_file, player_merge_keys, player_features)
df_mf_p = load_filtered(mf_file, player_merge_keys, player_features)
df_df_p = load_filtered(df_file, player_merge_keys, player_features)

merged_players = df_fw_p.merge(df_mf_p, on=player_merge_keys, how='outer')
merged_players = merged_players.merge(df_df_p, on=player_merge_keys, how='outer')
merged_players = merged_players.loc[:, ~merged_players.columns.duplicated()]
print(f"✅ Merged players shape: {merged_players.shape}")

# === LOAD, AGGREGATE AND MERGE TEAMS ===
def load_and_group_team(file, keys, features):
    df = pd.read_csv(file)
    df = df[[col for col in keys + features if col in df.columns]].copy()
    return df.groupby(keys, as_index=False).mean(numeric_only=True)

df_fw_t = load_and_group_team(fw_file, team_merge_keys, team_features)
df_mf_t = load_and_group_team(mf_file, team_merge_keys, team_features)
df_df_t = load_and_group_team(df_file, team_merge_keys, team_features)

merged_teams = df_fw_t.merge(df_mf_t, on=team_merge_keys, how='outer')
merged_teams = merged_teams.merge(df_df_t, on=team_merge_keys, how='outer')
merged_teams = merged_teams.loc[:, ~merged_teams.columns.duplicated()]
print(f"✅ Merged teams shape: {merged_teams.shape}")

# === SCALING: PLAYERS ===
exclude_players = ["Player", "Season", "Age", "Squad", "Pos"]
numeric_player_cols = [col for col in merged_players.columns if col not in exclude_players and pd.api.types.is_numeric_dtype(merged_players[col])]

player_scaler = MinMaxScaler(feature_range=(20, 100))
merged_players[numeric_player_cols] = player_scaler.fit_transform(merged_players[numeric_player_cols])

# === SCALING: TEAMS ===
exclude_teams = ["Squad", "Season"]
numeric_team_cols = [col for col in merged_teams.columns if col not in exclude_teams and pd.api.types.is_numeric_dtype(merged_teams[col])]

team_scaler = MinMaxScaler(feature_range=(20, 100))
merged_teams[numeric_team_cols] = team_scaler.fit_transform(merged_teams[numeric_team_cols])

# === SAVE FILES ===
merged_players.to_csv(
    "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/merged_players.csv",
    index=False
)

merged_teams.to_csv(
    "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/merged_teams.csv",
    index=False
)

print("🎉 All done: saved 'merged_players.csv' and 'merged_teams.csv'")


✅ Merged players shape: (15024, 35)
✅ Merged teams shape: (390, 24)
🎉 All done: saved 'merged_players.csv' and 'merged_teams.csv'


## Filtering and Verifying Player & Team Data for the 2023–2024 Season

This script filters the full player and team datasets to retain only entries from the 2023–2024 season. It performs the following tasks:

### Key Operations

- **Load Cleaned Datasets**  
  Reads `merged_players.csv` and `merged_teams.csv`, which contain scaled contribution metrics.

- **Missing Squad Checks**  
  Identifies player rows with missing or invalid `Squad` values in the 2023–2024 data.

- **Club-Specific Subsetting**  
  Verifies the presence of specific clubs (e.g., Manchester City) using a case-insensitive match.

- **Filtering by Season**  
  Filters the datasets to keep only 2023–2024 season entries for both players and teams.

- **Export Filtered Data**  
  Saves the season-specific datasets as:
  - `players_2023_2024.csv`
  - `teams_2023_2024.csv`

These filtered files are used in downstream similarity matching, role gap detection, and optimization modeling.


In [ ]:
import pandas as pd
import os

# === CONFIGURATION ===
input_players_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/merged_players.csv"
input_teams_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/merged_teams.csv"

output_dir = "/content/drive/MyDrive/masters thesis ali alhaj hassan/optimization"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# === LOAD DATA ===
players_df = pd.read_csv(input_players_file)
teams_df = pd.read_csv(input_teams_file)
# Check total player counts
print(f"🔢 Total rows: {players_df.shape[0]}")
print(f"🟢 Rows for 2023-2024: {players_df[players_df['Season'] == '2023-2024'].shape[0]}")

# Count per season
print(players_df["Season"].value_counts())

# Check if any NaNs in 2023-2024 players
print("❓ Missing values in 2023-2024:")
print(players_df[players_df["Season"] == "2023-2024"].isnull().sum())
# Filter all 2023-2024 players with missing or incorrect Squad
mask = (players_df["Season"] == "2023-2024") & (players_df["Squad"].isna() | (players_df["Squad"] == ""))
missing_squad_players = players_df[mask]
print(missing_squad_players[["Player", "Season", "Squad", "Squad_x", "Squad_y"]])
# Case-insensitive contains filter for 'Manchester City'
mc_players = players_df[
    (players_df["Season"] == "2023-2024") &
    (players_df["Squad"].str.contains("Manchester city", case=False, na=False))
]

print(f"✅ Found {len(mc_players)} Manchester City players for 2023-2024")
print(mc_players[["Player", "Season", "Squad"]])
import os

print("🧪 Verifying written output...")
players_2023_path = os.path.join(output_dir, "players_2023_2024.csv")
players_2023_df = pd.read_csv(players_2023_path)

print("✅ Manchester City players in written CSV:")
print(players_2023_df[players_2023_df["Squad"].str.contains("manchester city", case=False)])

# === FILTER FOR SEASON 2023-2024 ===
players_2023 = players_df[players_df["Season"] == "2023-2024"].copy()
teams_2023 = teams_df[teams_df["Season"] == "2023-2024"].copy()

# === SAVE FILTERED DATA ===
players_2023.to_csv(os.path.join(output_dir, "players_2023_2024.csv"), index=False)
teams_2023.to_csv(os.path.join(output_dir, "teams_2023_2024.csv"), index=False)

print("✅ Saved filtered players and teams for season 2023-2024.")


🔢 Total rows: 15024
🟢 Rows for 2023-2024: 3758
Season
2022-2023    3933
2023-2024    3758
2021-2022    3743
2020-2021    3590
Name: count, dtype: int64
❓ Missing values in 2023-2024:
Player                           0
Season                           0
Age_x                            2
Squad_x                          0
Gls_Contribution                 0
Sh_Contribution                  0
SoT_Contribution                 0
xG_Contribution                  0
G-xG_Contribution                0
Total_Player_Contribution_att    0
Pos_x                            0
Team_Att_Score                   0
Age_y                            2
Squad_y                          0
xAG_Contribution                 0
Ast_Contribution                 0
KP_Contribution                  0
PPA_Contribution                 0
PrgP_Contribution                0
Total_Player_Contribution_mid    0
Pos_y                            0
Team_Mid_Score                   0
Age                              2
Tkl_Contribu

## Preparing 2023–2024 Player Dataset for Optimization

This script filters and prepares the player dataset for the 2023–2024 season by:

- **Loading** the merged player dataset from previous preprocessing steps.
- **Cleaning** the `Season` column to ensure uniform formatting.
- **Filtering** rows to retain only players from the 2023–2024 season.
- **Removing duplicates** to avoid redundant entries.
- **Dropping auxiliary columns** such as `Squad_x`, `Squad_y`, `Age_x`, and `Age_y` if present due to merges.
- **Saving** the cleaned subset to `players_2023_2024.csv`, which will be used in the optimization phase.

This ensures that only clean, relevant, and season-specific player data is fed into the recruitment and squad planning model.


In [ ]:
import pandas as pd
import os

# === FILE PATHS ===
input_players = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/merged_players.csv"
output_dir = "/content/drive/MyDrive/masters thesis ali alhaj hassan/optimization"
os.makedirs(output_dir, exist_ok=True)

output_players = os.path.join(output_dir, "players_2023_2024.csv")

# === LOAD DATA ===
players_df = pd.read_csv(input_players)

# === CLEAN SEASON COLUMN ===
players_df["Season"] = players_df["Season"].astype(str).str.strip()

# === FILTER FOR 2023–2024 ===
players_2023 = players_df[players_df["Season"] == "2023-2024"].copy()

# === DROP DUPLICATES AND UNWANTED COLUMNS ===
players_2023 = players_2023.drop_duplicates()

# Drop Squad_x and Squad_y if they exist
players_2023 = players_2023.drop(columns=[col for col in ["Squad_x", "Squad_y","Age_x","Age_y"] if col in players_2023.columns])

# === SAVE TO FILE ===
players_2023.to_csv(output_players, index=False)

print(f"✅ Saved {len(players_2023)} players to: {output_players}")


✅ Saved 3758 players to: /content/drive/MyDrive/masters thesis ali alhaj hassan/optimization/players_2023_2024.csv


## Analyzing Top Team Performances by Line – Season 2023–2024

This script provides a summary of the top 5 performing teams in the 2023–2024 season based on three key tactical dimensions:

- **Defensive Strength:** Teams are ranked by the `Team_Def_Score`, reflecting their performance in minimizing goals conceded and excelling in defensive metrics.
- **Midfield Performance:** Rankings are based on the `Team_Mid_Score`, a composite index derived from metrics like progressive passes, key passes, and expected assists.
- **Attacking Efficiency:** Teams are sorted by `Team_Att_Score`, which captures goal-scoring and xG-related effectiveness.

Each category is sorted in descending order and the top 5 teams are printed with their corresponding scores for reporting or diagnostic purposes.


In [ ]:
import pandas as pd

# Load team data for 2023-2024
teams_df = pd.read_csv("/content/drive/MyDrive/masters thesis ali alhaj hassan/optimization/teams_2023_2024.csv")

# Sort and display top 5 teams by each category
top_def = teams_df.sort_values(by="Team_Def_Score", ascending=False).head(5)
top_mid = teams_df.sort_values(by="Team_Mid_Score", ascending=False).head(5)
top_att = teams_df.sort_values(by="Team_Att_Score", ascending=False).head(5)

# === PRINT REPORT ===
print("🏰 Top 5 Teams Defensively (2023–2024):")
for i, row in top_def.iterrows():
    print(f"{i+1}. {row['Squad']} — Defense Score: {row['Team_Def_Score']:.2f}")

print("\n🎯 Top 5 Teams in Midfield (2023–2024):")
for i, row in top_mid.iterrows():
    print(f"{i+1}. {row['Squad']} — Midfield Score: {row['Team_Mid_Score']:.2f}")

print("\n🔥 Top 5 Teams Attacking (2023–2024):")
for i, row in top_att.iterrows():
    print(f"{i+1}. {row['Squad']} — Attack Score: {row['Team_Att_Score']:.2f}")


🏰 Top 5 Teams Defensively (2023–2024):
42. Inter — Defense Score: 91.00
50. Leverkusen — Defense Score: 83.49
75. Real Madrid — Defense Score: 77.14
3. Arsenal — Defense Score: 69.26
69. Nice — Defense Score: 69.26

🎯 Top 5 Teams in Midfield (2023–2024):
52. Liverpool — Midfield Score: 96.69
3. Arsenal — Midfield Score: 90.53
58. Manchester City — Midfield Score: 90.45
10. Bayern Munich — Midfield Score: 87.87
87. Tottenham — Midfield Score: 83.74

🔥 Top 5 Teams Attacking (2023–2024):
10. Bayern Munich — Attack Score: 96.01
58. Manchester City — Attack Score: 93.36
52. Liverpool — Attack Score: 90.78
42. Inter — Attack Score: 88.03
3. Arsenal — Attack Score: 86.07


## Integrating Market Value Data into Player Dataset – Season 2023–2024

This script augments the existing player-level dataset for the 2023–2024 season by merging it with market value information sourced from Transfermarkt.

**Key Steps:**
- Loads cleaned performance data and market value data.
- Strips whitespace and standardizes player name columns.
- Performs an inner merge on the `Player` column to associate each player with their market value.
- Renames the market value column for consistency (`MarketValue`).
- Deduplicates the dataset, keeping only the first record per player.
- Saves the updated dataset back to the same file.

This merged dataset serves as a foundation for further optimization or financial-performance analysis.


In [ ]:
import pandas as pd

# === CONFIGURATION ===
player_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/optimization/players_2023_2024.csv"
market_value_file = "/content/drive/MyDrive/masters thesis ali alhaj hassan/data_scraping_and_preperation/transfermarkt/transfermarkt_data_2023_2024/all_leagues_combined_2023_2024.csv"

# === LOAD FILES ===
players_df = pd.read_csv(player_file)
market_df = pd.read_csv(market_value_file)

# Clean column names
players_df.columns = players_df.columns.str.strip()
market_df.columns = market_df.columns.str.strip()

# Clean Player names
players_df["Player"] = players_df["Player"].astype(str).str.strip()
market_df["Player"] = market_df["Player"].astype(str).str.strip()

# === MERGE ON 'Player' ===
players_df = pd.merge(
    players_df,
    market_df[["Player", "Market Value (€)"]],
    on="Player",
    how="inner"
)

# Rename for consistency
players_df.rename(columns={"Market Value (€)": "MarketValue"}, inplace=True)

# === KEEP ONLY ONE ROW PER PLAYER ===
players_df = players_df.drop_duplicates(subset=["Player"], keep="first")

# === SAVE ===
players_df.to_csv(player_file, index=False)

print(f"✅ File updated: kept first row per player. Final count: {len(players_df)}")


✅ File updated: kept first row per player. Final count: 2344


## Generating Inverse-Normalized Team Needs Scores (2023–2024)

This script transforms raw team performance metrics into a normalized representation of team needs, where higher values indicate more urgent needs.

**Key Operations:**
- Loads the 2023–2024 team-level performance dataset.
- Excludes non-numeric columns (`Squad`, `Season`) from scaling.
- Applies MinMaxScaler on the **reversed values** of performance metrics so that:
  - Low performance scores become **high needs**.
  - Final scaled values are in the range [20, 100], where **100 represents the most severe need**.
- Saves the output to a CSV file (`teams_needs_2023_2024.csv`), which will be used in optimization models.

This transformation enables the modeling of team deficiencies in defense, midfield, and attack, providing a foundation for role-based recruitment strategies.


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# === Load the merged_teams.csv file ===
teams_df = pd.read_csv("/content/drive/MyDrive/masters thesis ali alhaj hassan/optimization/teams_2023_2024.csv")
teams_df.columns = teams_df.columns.str.strip()

# === Identify numeric feature columns only (exclude Squad and Season) ===
non_feature_cols = ["Squad", "Season"]
numeric_cols = [col for col in teams_df.columns if col not in non_feature_cols]

# === Apply MinMaxScaler but reversed: 100 = worst, 20 = best ===
# Multiply by -1 to reverse, then scale, then flip range (100=worst)
scaler = MinMaxScaler(feature_range=(20, 100))

# Reverse the values for proper scaling (low raw score ➜ high need)
reversed_values = -teams_df[numeric_cols]

# Fit scaler on reversed values
scaled_needs = scaler.fit_transform(reversed_values)

# Create output DataFrame
need_df = teams_df[["Squad", "Season"]].copy()
need_df[numeric_cols] = scaled_needs

# === Save to file ===
need_df.to_csv("/content/drive/MyDrive/masters thesis ali alhaj hassan/optimization/teams_needs_2023_2024.csv", index=False)

print("✅ Saved reversed MinMax scaled team needs to 'needs.csv'")


✅ Saved reversed MinMax scaled team needs to 'needds.csv'
